In [1]:
import pandas as pd
import requests
import boto3
from utils import *
import datetime
from consts import *

**EXTRACTION**

GET SUMMONER PUUID BY SUMMONER NAME AND TAGLINE

In [2]:
endpoint = f"https://{SERVERS['AMERICAS']}/riot/account/v1/accounts/by-riot-id/{SUMM_NAME}/{TAG_LINE}"

res = requests.get(endpoint, params=PARAMS)
data = res.json()

puuid = data['puuid']

df_summoner = pd.json_normalize(data)

df_summoner

,puuid,gameName,tagLine
0,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,Girebaomu,Girev


GET SUMMONER DATA BY PUUID

In [3]:
endpoint = f"https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"

res = requests.get(endpoint, params=PARAMS)
data = dict(res.json())

summ_id = data['id']

data.pop('puuid')

df_summoner = pd.concat([df_summoner, pd.json_normalize(data)], axis=1)

df_summoner = df_summoner.drop(columns=['profileIconId'])

df_summoner

,puuid,gameName,tagLine,id,accountId,revisionDate,summonerLevel
0,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,Girebaomu,Girev,bV9iI7bt1MqN90vlAKyFu2FQaXwbcBqHFIoHXfuL7D9lJ8...,MFgKEYpE6CssSjNUETb2DLA1xA5dQg_t8qcWs6r8c24jl-...,1738031754784,82


GET CHAMPIONS LIST

In [4]:
champions_endpoint = "https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json"

data = requests.get(champions_endpoint).json()

df_champions = pd.DataFrame(data['data'])

df_champions = df_champions.T

df_champions.head(1)

,version,id,key,name,title,blurb,info,image,tags,partype,stats
Aatrox,14.22.1,Aatrox,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...",[Fighter],Blood Well,"{'hp': 650, 'hpperlevel': 114, 'mp': 0, 'mpper..."


GET MASTERY CHAMPIONS BY SUMMONER PUUID

In [5]:
endpoint = f"https://la2.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"

res = requests.get(endpoint, params=PARAMS)
data = res.json()

df_champions_mastery = pd.json_normalize(data)

df_champions_mastery.head()


,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,markRequiredForNextLevel,tokensEarned,championSeasonMilestone,milestoneGrades,nextSeasonMilestone.requireGradeCounts.S-,nextSeasonMilestone.rewardMarks,nextSeasonMilestone.bonus,nextSeasonMilestone.totalGamesRequires,nextSeasonMilestone.requireGradeCounts.A-
0,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,222,9,70690,1738025061000,6090,4910,2,6,2,[B-],1.0,2,False,1,NaN
1,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,145,5,27320,1738031755000,5720,4280,1,3,2,[],1.0,2,False,1,NaN
2,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,202,5,23299,1738027902000,1699,8301,1,6,5,[],2.0,1,True,2,NaN
3,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,112,4,22135,1717646158000,9535,-535,1,0,0,NaN,NaN,1,False,1,1.0
4,k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbN...,22,4,13198,1736018076000,598,8402,1,0,0,NaN,NaN,1,False,1,1.0


GET THE LAST 20 MATCHES:
1- FIRST I GET THE MATCHES IDs.
2- SINCE FROM THOSE IDs, I CAN GET MATCHES DATA 

In [6]:
endpoint = f"https://{SERVERS['AMERICAS']}/lol/match/v5/matches/by-puuid/{puuid}/ids"

res = requests.get(endpoint, params=PARAMS)
matches_id = res.json()

matches_id

matches = []
for matchId in matches_id:
    endpoint = f"https://{SERVERS['AMERICAS']}/lol/match/v5/matches/{matchId}"
    
    res = requests.get(endpoint, params=PARAMS)
    matches.append(res.json())
    
df_matches = pd.json_normalize(matches)

In [7]:
df_matches.head(1)

,metadata.dataVersion,metadata.matchId,metadata.participants,info.endOfGameResult,info.gameCreation,info.gameDuration,info.gameEndTimestamp,info.gameId,info.gameMode,info.gameName,info.gameStartTimestamp,info.gameType,info.gameVersion,info.mapId,info.participants,info.platformId,info.queueId,info.teams,info.tournamentCode
0,2,LA2_1481563912,[wyFAf3t8_twxyjjyTKXt5JywvvUqCFDPzQc7151CTgZGY...,GameComplete,1738029700931,1899,1738031750255,1481563912,CLASSIC,teambuilder-match-1481563912,1738029850869,MATCHED_GAME,15.2.652.1536,11,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...",LA2,420,"[{'bans': [{'championId': 99, 'pickTurn': 1}, ...",


MATCHES DETAILS

In [8]:
df_matches_details = df_matches[["metadata.matchId", "metadata.participants","info.participants","info.teams"]]

df_matches = df_matches.drop(columns=["metadata.participants","info.participants","info.teams"])

In [9]:
df_matches_details.head(1)

,metadata.matchId,metadata.participants,info.participants,info.teams
0,LA2_1481563912,[wyFAf3t8_twxyjjyTKXt5JywvvUqCFDPzQc7151CTgZGY...,"[{'PlayerScore0': 0, 'PlayerScore1': 0, 'Playe...","[{'bans': [{'championId': 99, 'pickTurn': 1}, ..."


GET SUMMONER RANKING BY SUMMONER ID

In [10]:
endpoint = f"https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}"

res = requests.get(endpoint, params=PARAMS)

data = res.json()

df_ranking = pd.json_normalize(data)

In [11]:
df_ranking.head()

,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,232695b6-e379-4d90-8169-9a32c4fb12da,RANKED_SOLO_5x5,PLATINUM,IV,bV9iI7bt1MqN90vlAKyFu2FQaXwbcBqHFIoHXfuL7D9lJ8...,63,8,4,False,False,True,False


GET ITEMS

In [12]:
endpoint = 'https://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/item.json'

res = requests.get(endpoint)
data = res.json()

item_list=[]
for id, data in data['data'].items():
    item={
        'id':id,
        'name':data['name'],
        'description':data['description'],
        'colloq':data['colloq'],
        'plaintext':data['plaintext'],
        'into':data.get('into'),
        'image':data['image'],
        'gold':data['gold'],
        'tags':data['tags'],
        'maps':data['maps'],
        'stats':data['stats']
    }
    item_list.append(item)
    
df_items = pd.DataFrame(item_list)

In [13]:
df_items.head(1)

,id,name,description,colloq,plaintext,into,image,gold,tags,maps,stats
0,1001,Boots,<mainText><stats><attention>25</attention> Mov...,;,Slightly increases Move Speed,"[3005, 3047, 3006, 3009, 3010, 3020, 3111, 311...","{'full': '1001.png', 'sprite': 'item0.png', 'g...","{'base': 300, 'purchasable': True, 'total': 30...",[Boots],"{'11': True, '12': True, '21': True, '22': Fal...",{'FlatMovementSpeedMod': 25}


**LOAD RAW DATA TO DATA LAKE (S3)**

CREATING S3 CLIENT

In [ ]:
s3_client = boto3.client('s3', region_name=AWS_REGION_NAME)

CREATING BUCKET

In [ ]:
#CREATING A BUCKET
s3_client.create_bucket(ACL='private', Bucket='mylolapibucket1')

LOADING DATA

In [ ]:
#PATH
s3_path = f"lol_data/{datetime.date.today().strftime('%Y-%m-%d')}"

upload_data_to_s3(s3_client, df_summoner, f"{s3_path}/summoner.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_ranking, f"{s3_path}/ranking.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_champions, f"{s3_path}/champions.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_champions_mastery, f"{s3_path}/champions_mastery.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_matches, f"{s3_path}/matches.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_matches_details, f"{s3_path}/matches_details.parquet", "mylolapibucket1")
upload_data_to_s3(s3_client, df_items, f"{s3_path}/items.parquet", "mylolapibucket1")

**TRANSFORMATION**

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, explode, concat_ws, round, regexp_replace, when, monotonically_increasing_id
import findspark

CREATING SPARK SESSION

In [15]:
findspark.init()
spark_session = SparkSession.builder \
                                    .appName("SUMMONERS_PIPELINE") \
                                    .config("spark.jars", JDBC_PATH) \
                                    .config("spark.driver.memory", "12g") \
                                    .getOrCreate()

GETTING PARQUETS FROM DATA LAKE (S3)

In [103]:
df_summoner_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/summoner.parquet"))
df_ranking_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/ranking.parquet"))
df_champions_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions.parquet"))
df_champions_mastery_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/champions_mastery.parquet"))
df_matches_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches.parquet"))
df_matches_details_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/matches_details.parquet"))
df_items_spark = spark_session.read.parquet(download_files_from_s3(s3_client, 'mylolapibucket1', f"{s3_path}/items.parquet"))

SUMMONER TRANSFORM

In [20]:
#JOIN SUMMONER DF AND RANKING DF
df_summoner_spark = df_summoner_spark.join(df_ranking_spark, on=df_summoner_spark["id"] == df_ranking_spark["summonerId"])
df_summoner_spark = df_summoner_spark.drop("id")

#CONVERT UNIX TIMESTAMP TO DATE
df_summoner_spark = df_summoner_spark.withColumn("revisionDate", from_unixtime(col("revisionDate") / 1000).cast("timestamp"))

columns = df_summoner_spark.columns

summoner_puuid = columns.index("puuid")
summoner_summonerId = columns.index("summonerId")

columns[summoner_puuid], columns[summoner_summonerId] = columns[summoner_summonerId], columns[summoner_puuid]

df_summoner_spark = df_summoner_spark.select(columns)

df_summoner_spark = df_summoner_spark.withColumn("veteran", when(col("veteran") == "True", 1).otherwise(0).cast("long"))
df_summoner_spark = df_summoner_spark.withColumn("inactive", when(col("inactive") == "True", 1).otherwise(0).cast("long"))
df_summoner_spark = df_summoner_spark.withColumn("freshBlood", when(col("freshBlood") == "True", 1).otherwise(0).cast("long"))
df_summoner_spark = df_summoner_spark.withColumn("hotStreak", when(col("hotStreak") == "True", 1).otherwise(0).cast("long"))

#ORDERING
df_summoner_spark = df_summoner_spark.select("puuid","summonerId","accountId","leagueId","gameName","tagLine",
                                             "summonerLevel","queueType","tier","rank","leaguePoints","wins",
                                             "losses","veteran","inactive","freshBlood","hotStreak","revisionDate")

In [21]:
df_summoner_spark.show()

+--------------------+--------------------+--------------------+--------------------+---------+-------+-------------+---------------+--------+----+------------+----+------+-------+--------+----------+---------+-------------------+
|               puuid|          summonerId|           accountId|            leagueId| gameName|tagLine|summonerLevel|      queueType|    tier|rank|leaguePoints|wins|losses|veteran|inactive|freshBlood|hotStreak|       revisionDate|
+--------------------+--------------------+--------------------+--------------------+---------+-------+-------------+---------------+--------+----+------------+----+------+-------+--------+----------+---------+-------------------+
|k2aD9VEYaqqcnXwHI...|bV9iI7bt1MqN90vlA...|MFgKEYpE6CssSjNUE...|232695b6-e379-4d9...|Girebaomu|  Girev|           82|RANKED_SOLO_5x5|PLATINUM|  IV|          63|   8|     4|      0|       0|         1|        0|2025-01-27 23:35:54|
+--------------------+--------------------+--------------------+------------

CHAMPIONS TRANSFORM

In [22]:
#LIST TO STRING
df_champions_spark = df_champions_spark.withColumn("tags", concat_ws(", ", col("tags")))

#EXPLODE INFO FIELD
df_champions_spark = df_champions_spark.select("*",
                                                 col("info.attack").alias("attack"),
                                                 col("info.defense").alias("defense"),
                                                 col("info.magic").alias("magic"),
                                                 col("info.difficulty").alias("difficulty"),
                                                 col("stats.hp").alias("hp"),
                                                 col("stats.hpperlevel").alias("hpPerLevel"),
                                                 col("stats.mp").alias("mp"),
                                                 col("stats.mpperlevel").alias("mpPerLevel"),
                                                 col("stats.movespeed").alias("moveSpeed"),
                                                 col("stats.armor").alias("armor"),
                                                 col("stats.armorperlevel").alias("armorPerLevel"),
                                                 col("stats.spellblock").alias("spellBlock"),
                                                 col("stats.spellblockperlevel").alias("spellBlockPerLevel"),
                                                 col("stats.attackrange").alias("attackRange"),
                                                 col("stats.hpregen").alias("hpRegen"),
                                                 col("stats.hpregenperlevel").alias("hpRegenPerLevel"),
                                                 col("stats.mpregen").alias("mpRegen"),
                                                 col("stats.mpregenperlevel").alias("mpRegenPerLevel"),
                                                 col("stats.crit").alias("crit"),
                                                 col("stats.critperlevel").alias("critPerLevel"),
                                                 col("stats.attackdamage").alias("attackDamage"),
                                                 col("stats.attackdamageperlevel").alias("attackDamagePerLevel"),
                                                 col("stats.attackspeed").alias("attackSpeed"),
                                                 col("stats.attackspeedperlevel").alias("attackSpeedPerLevel"))

df_champions_spark = df_champions_spark.drop("info", "image", "stats", "id", "version")

df_champions_spark = df_champions_spark.withColumnRenamed("key", "championId")

df_champions_spark = df_champions_spark.withColumn("partype", when(col("partype") == "", "None").otherwise(col("partype")))

df_champions_spark = df_champions_spark.withColumn("championId", col("championId").cast("long"))

In [23]:
df_champions_spark.show(1,truncate=False)

+----------+------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----------+------+-------+-----+----------+---+----------+---+----------+---------+-----+-------------+----------+------------------+-----------+-------+---------------+-------+---------------+----+------------+------------+--------------------+-----------+-------------------+
|championId|name  |title           |blurb                                                                                                                                                                                                                                                          |tags   |partype   |attack|defense|magic|difficulty|hp |hpPerLevel|mp |mpPerLevel|moveSpeed|armor|armorPerLevel|spellBlock|spellBlo

CHAMPIONS MASTERY TRANSFORM

In [25]:
df_champions_mastery_spark = df_champions_mastery_spark.withColumn("lastPlayTime", from_unixtime(col("lastPlayTime") / 1000).cast("timestamp"))

df_champions_mastery_spark = df_champions_mastery_spark.drop("championPointsSinceLastLevel",
                                                             "championPointsUntilNextLevel",
                                                             "markRequiredForNextLevel",
                                                             "tokensEarned",
                                                             "championSeasonMilestone",
                                                             "nextSeasonMilestone.requireGradeCounts.B-",
                                                             "nextSeasonMilestone.requireGradeCounts.C-",
                                                             "nextSeasonMilestone.rewardMarks",
                                                             "nextSeasonMilestone.bonus",
                                                             "nextSeasonMilestone.rewardConfig.rewardValue",
                                                             "nextSeasonMilestone.rewardConfig.rewardType",
                                                             "nextSeasonMilestone.rewardConfig.maximumReward",
                                                             "nextSeasonMilestone.totalGamesRequires",
                                                             "milestoneGrades",
                                                             "nextSeasonMilestone.requireGradeCounts.A-")

In [26]:
df_champions_mastery_spark.show(1, truncate=False)

+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+-----------------------------------------+
|puuid                                                                         |championId|championLevel|championPoints|lastPlayTime       |nextSeasonMilestone.requireGradeCounts.S-|
+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+-----------------------------------------+
|k2aD9VEYaqqcnXwHIT1gTqsprvoOAOFHej8m6Dgzv8DsbNR7pOJAQLghwwWt0MHtKkuAzmVSt6jiBQ|222       |9            |70690         |2025-01-27 21:44:21|1.0                                      |
+------------------------------------------------------------------------------+----------+-------------+--------------+-------------------+-----------------------------------------+
only showing top 1 row



MATCHES TRANSFORM

In [27]:
df_matches_spark = df_matches_spark.withColumnsRenamed({"metadata.dataVersion":"dataVersion",
                                                        "metadata.matchId":"matchId",
                                                        "info.endOfGameResult":"endOfGameResult",
                                                        "info.gameCreation":"gameCreation",
                                                        "info.gameEndTimestamp":"gameEndTimestamp",
                                                        "info.gameDuration":"gameDuration",
                                                        "info.gameId":"gameId",
                                                        "info.gameMode":"gameMode",
                                                        "info.gameName":"gameName",
                                                        "info.gameStartTimestamp":"gameStartTimestamp",
                                                        "info.gameType":"gameType",
                                                        "info.gameVersion":"gameVersion",
                                                        "info.mapId":"mapId",
                                                        "info.platformId":"platformId",
                                                        "info.queueId":"queueId",
                                                        "info.tournamentCode":"tournamentCode",
                                                        })

df_matches_spark = df_matches_spark.withColumn("gameCreation", from_unixtime(col("gameCreation") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameEndTimestamp", from_unixtime(col("gameEndTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameStartTimestamp", from_unixtime(col("gameStartTimestamp") / 1000).cast("timestamp"))
df_matches_spark = df_matches_spark.withColumn("gameDuration", round((col("gameDuration") / 60).cast("double"), 2))

df_matches_spark = df_matches_spark.drop("tournamentCode", "dataVersion")

df_matches_spark = df_matches_spark.select("matchId", 
                                           "gameId",
                                           "endOfGameResult",
                                           "gameCreation", 
                                           "gameStartTimestamp", 
                                           "gameEndTimestamp", 
                                           "gameDuration",
                                           "gameMode",
                                           "gameName",
                                           "gameType",
                                           "mapId",
                                           "platformId",
                                           "queueId",
                                           "gameVersion")

In [30]:
df_matches_spark.show(5,truncate=False)

+--------------+----------+---------------+-------------------+-------------------+-------------------+------------+--------+----------------------------+------------+-----+----------+-------+-------------+
|matchId       |gameId    |endOfGameResult|gameCreation       |gameStartTimestamp |gameEndTimestamp   |gameDuration|gameMode|gameName                    |gameType    |mapId|platformId|queueId|gameVersion  |
+--------------+----------+---------------+-------------------+-------------------+-------------------+------------+--------+----------------------------+------------+-----+----------+-------+-------------+
|LA2_1481563912|1481563912|GameComplete   |2025-01-27 23:01:40|2025-01-27 23:04:10|2025-01-27 23:35:50|31.65       |CLASSIC |teambuilder-match-1481563912|MATCHED_GAME|11   |LA2       |420    |15.2.652.1536|
|LA2_1481540181|1481540181|GameComplete   |2025-01-27 21:58:16|2025-01-27 21:59:34|2025-01-27 22:31:35|32.02       |CLASSIC |teambuilder-match-1481540181|MATCHED_GAME|11   

MATCHES DETAILS TRANSFORM

In [31]:
df_matches_details_spark = df_matches_details_spark.withColumnsRenamed({"metadata.matchId":"matchId",
                                                                        "metadata.participants":"participant_id",
                                                                        "info.participants":"participant_info",
                                                                        "info.teams":"infoTeams"})

#METADATA PARTICIPANTS EXPLODE
participants = df_matches_details_spark.select("matchId", "participant_id")
participants = participants.withColumn("participant_id", explode("participant_id"))

#PARTICIPANTS INFO EXPLODE
participants_info = df_matches_details_spark.select("participant_info")
participants_info = participants_info.withColumn("participant_info", explode("participant_info"))
participants_info = participants_info.select("*", col("participant_info.puuid").alias("puuid"),
                                                  col("participant_info.summonerId").alias("summonerId"),
                                                  col("participant_info.teamId").alias("teamId"),
                                                  col("participant_info.summonerName").alias("summonerName"),
                                                  col("participant_info.summonerLevel").alias("summonerLevel"),
                                                  col("participant_info.win").alias("win"),
                                                  col("participant_info.lane").alias("lane"),
                                                  col("participant_info.role").alias("role"),
                                                  col("participant_info.teamPosition").alias("teamPosition"),
                                                  col("participant_info.individualPosition").alias("individualPosition"),
                                                  col("participant_info.championId").alias("championId"),
                                                  col("participant_info.championName").alias("championName"),
                                                  col("participant_info.champLevel").alias("champLevel"),
                                                  col("participant_info.kills").alias("kills"),
                                                  col("participant_info.deaths").alias("deaths"),
                                                  col("participant_info.assists").alias("assists"),
                                                  col("participant_info.totalMinionsKilled").alias("totalMinionsKilled"),
                                                  col("participant_info.goldEarned").alias("goldEarned"),
                                                  col("participant_info.goldSpent").alias("goldSpent"),
                                                  col("participant_info.timePlayed").alias("timePlayed"),
                                                  col("participant_info.doubleKills").alias("doubleKills"),
                                                  col("participant_info.tripleKills").alias("tripleKills"),
                                                  col("participant_info.quadraKills").alias("quadraKills"),
                                                  col("participant_info.pentaKills").alias("pentaKills"),
                                                  col("participant_info.killingSprees").alias("killingSprees"),
                                                  col("participant_info.visionScore").alias("visionScore"),
                                                  col("participant_info.dragonKills").alias("dragonKills"),
                                                  col("participant_info.turretKills").alias("turretKills"),
                                                  col("participant_info.inhibitorKills").alias("inhibitorKills"),
                                                  col("participant_info.nexusKills").alias("nexusKills"),
                                                  col("participant_info.inhibitorTakedowns").alias("inhibitorTakedowns"),
                                                  col("participant_info.inhibitorsLost").alias("inhibitorsLost"),
                                                  col("participant_info.objectivesStolen").alias("objectivesStolen"),
                                                  col("participant_info.item0").alias("item0"),
                                                  col("participant_info.item1").alias("item1"),
                                                  col("participant_info.item2").alias("item2"),
                                                  col("participant_info.item3").alias("item3"),
                                                  col("participant_info.item4").alias("item4"),
                                                  col("participant_info.item5").alias("item5"),
                                                  col("participant_info.item6").alias("item6"))
participants_info = participants_info.withColumn("timePlayed", round(col("participant_info.timePlayed") / 60, 2))
participants_info = participants_info.withColumnRenamed("timePlayed", "timePlayed(mins)")
participants_info = participants_info.drop("participant_info")

#EXPLODE INFOTEAMS TO GET BANS
info_teams = df_matches_details_spark.select("infoTeams")
bans = info_teams.withColumn("bans", col("infoTeams.bans").alias("bans"))
bans = bans.withColumn("bans", explode("bans"))
bans = bans.withColumn("banChampionId", col("bans.championId").alias("championId"))
bans = bans.withColumn("banChampionId", explode("banChampionId"))
bans = bans.drop("infoTeams","bans")



#JOIN MATCHES_DETAILS AND BANS
participants = participants.withColumn("id", monotonically_increasing_id())
participants_info = participants_info.withColumn("id", monotonically_increasing_id())
bans = bans.withColumn("id", monotonically_increasing_id())
df_matches_details_spark = df_matches_details_spark.withColumn("id", monotonically_increasing_id())

df_matches_details_spark = df_matches_details_spark.drop("matchId", "participant_id", "participant_info", "infoTeams")

df_matches_details_spark = df_matches_details_spark.join(participants, on="id")\
                                                   .join(participants_info, on="id")\
                                                   .join(bans, on="id")
                                                   
df_matches_details_spark = df_matches_details_spark.withColumnRenamed("participant_id", "participantId")

df_matches_details_spark = df_matches_details_spark.withColumn("win", when(col("win")=="True", 1 ).otherwise(0).cast("long"))
cols_to_select = [
    'puuid',
    'summonerId',
    'participantId',
    'matchId',
    'teamId',
    'summonerName',
    'summonerLevel',
    'win',
    'lane',
    'role',
    'teamPosition',
    'individualPosition',
    'championId',
    'championName',
    'champLevel',
    'kills',
    'deaths',
    'assists',
    'banChampionId',
    'totalMinionsKilled',
    'goldEarned',
    'goldSpent',
    'timePlayed(mins)',
    'doubleKills',
    'tripleKills',
    'quadraKills',
    'pentaKills',
    'killingSprees',
    'visionScore',
    'dragonKills',
    'turretKills',
    'inhibitorKills',
    'nexusKills',
    'inhibitorTakedowns',
    'inhibitorsLost',
    'objectivesStolen',
    'item0',
    'item1',
    'item2',
    'item3',
    'item4',
    'item5',
    'item6'
]

df_matches_details_spark = df_matches_details_spark.select(cols_to_select)

In [37]:
df_matches_details_spark.show(10, truncate=False)

+------------------------------------------------------------------------------+----------------------------------------------------------+------------------------------------------------------------------------------+--------------+------+----------------+-------------+---+------+-------+------------+------------------+----------+------------+----------+-----+------+-------+-------------+------------------+----------+---------+----------------+-----------+-----------+-----------+----------+-------------+-----------+-----------+-----------+--------------+----------+------------------+--------------+----------------+-----+-----+-----+-----+-----+-----+-----+
|puuid                                                                         |summonerId                                                |participantId                                                                 |matchId       |teamId|summonerName    |summonerLevel|win|lane  |role   |teamPosition|individualPosition|championId|c

JOIN MATCHES DATAFRAMES

In [33]:
df_matches_join = df_matches_spark.join(df_matches_details_spark, on="matchId")

In [36]:
df_matches_join.show(10, truncate=False)

+--------------+----------+---------------+-------------------+-------------------+-------------------+------------+--------+----------------------------+------------+-----+----------+-------+-------------+------------------------------------------------------------------------------+----------------------------------------------------------+------------------------------------------------------------------------------+------+----------------+-------------+---+------+-------+------------+------------------+----------+------------+----------+-----+------+-------+-------------+------------------+----------+---------+----------------+-----------+-----------+-----------+----------+-------------+-----------+-----------+-----------+--------------+----------+------------------+--------------+----------------+-----+-----+-----+-----+-----+-----+-----+
|matchId       |gameId    |endOfGameResult|gameCreation       |gameStartTimestamp |gameEndTimestamp   |gameDuration|gameMode|gameName           

MATCHES DIMENSION

In [35]:
dim_matches = df_matches_join.select("puuid",
                                     "matchId",
                                     "gameId",
                                     "teamId",
                                     "lane",
                                     "role",
                                     "teamPosition",
                                     "individualPosition",
                                     "endOfGameResult",
                                     "gameCreation",
                                     "gameStartTimestamp",
                                     "gameEndTimestamp",
                                     "gameMode",
                                     "gameName",
                                     "gameType",
                                     "mapId",
                                     "platformId",
                                     "queueId",
                                     "gameVersion")

In [80]:
dim_matches.show(1, truncate=False)

+------------------------------------------------------------------------------+--------------+----------+------+----+-------+------------+------------------+---------------+-------------------+-------------------+-------------------+--------+----------------------------+------------+-----+----------+-------+--------------+
|puuid                                                                         |matchId       |gameId    |teamId|lane|role   |teamPosition|individualPosition|endOfGameResult|gameCreation       |gameStartTimestamp |gameEndTimestamp   |gameMode|gameName                    |gameType    |mapId|platformId|queueId|gameVersion   |
+------------------------------------------------------------------------------+--------------+----------+------+----+-------+------------+------------------+---------------+-------------------+-------------------+-------------------+--------+----------------------------+------------+-----+----------+-------+--------------+
|lmY9N2GCcOkNkiEn2FCEc

FACT MATCHES

In [12]:
fact_matches = df_matches_join.select("puuid",
                                      "matchId",
                                      "championId",
                                      "gameDuration",
                                      "win",
                                      "banChampionId",
                                      "kills",
                                      "deaths",
                                      "assists",
                                      "champLevel",
                                      "totalMinionsKilled",
                                      "goldEarned",
                                      "goldSpent",
                                      "visionScore",
                                      "dragonKills",
                                      "turretKills",
                                      "doubleKills",
                                      "tripleKills",
                                      "quadraKills",
                                      "pentaKills",
                                      "killingSprees",
                                      "inhibitorKills",
                                      "nexusKills",
                                      "inhibitorTakedowns",
                                      "inhibitorsLost",
                                      "objectivesStolen",
                                      "item0",
                                      "item1",
                                      "item2",
                                      "item3",
                                      "item4",
                                      "item5",
                                      "item6"
                                      )

In [82]:
fact_matches.show(1, truncate=False)

+------------------------------------------------------------------------------+--------------+----------+------------+---+-------------+-----+------+-------+----------+------------------+----------+---------+-----------+-----------+-----------+-----------+-----------+-----------+----------+-------------+--------------+----------+------------------+--------------+----------------+-----+-----+-----+-----+-----+-----+-----+
|puuid                                                                         |matchId       |championId|gameDuration|win|banChampionId|kills|deaths|assists|champLevel|totalMinionsKilled|goldEarned|goldSpent|visionScore|dragonKills|turretKills|doubleKills|tripleKills|quadraKills|pentaKills|killingSprees|inhibitorKills|nexusKills|inhibitorTakedowns|inhibitorsLost|objectivesStolen|item0|item1|item2|item3|item4|item5|item6|
+------------------------------------------------------------------------------+--------------+----------+------------+---+-------------+-----+-----

ITEMS TRANSFORM

In [13]:
df_items_spark = df_items_spark.select("id", "name", "description", "plaintext", "into", "gold", "stats")

df_items_spark = df_items_spark.select("*",
                                       col("gold.base").alias("goldBase"),
                                       col("gold.total").alias("goldTotal"),
                                       col("gold.sell").alias("goldSell"),
                                       col("stats.FlatArmorMod").alias("statFlatArmorMod"),
                                       col("stats.FlatCritChanceMod").alias("statFlatCritChanceMod"),
                                       col("stats.FlatHPPoolMod").alias("statFlatHPPoolMod"),
                                       col("stats.FlatHPRegenMod").alias("statFlatHPRegenMod"),
                                       col("stats.FlatMPPoolMod").alias("statFlatMPPoolMod"),
                                       col("stats.FlatMagicDamageMod").alias("statFlatMagicDamageMod"),
                                       col("stats.FlatMovementSpeedMod").alias("statFlatMovementSpeedMod"),
                                       col("stats.FlatPhysicalDamageMod").alias("statFlatPhysicalDamageMod"),
                                       col("stats.FlatSpellBlockMod").alias("statFlatSpellBlockMod"),
                                       col("stats.PercentAttackSpeedMod").alias("statPercentAttackSpeedMod"),
                                       col("stats.PercentLifeStealMod").alias("statPercentLifeStealMod"),
                                       col("stats.PercentMovementSpeedMod").alias("statPercentMovementSpeedMod"),
                                       )

df_items_spark = df_items_spark.na.fill(0,subset=["statFlatArmorMod",
                                                "statFlatCritChanceMod",
                                                "statFlatHPPoolMod",
                                                "statFlatHPRegenMod",
                                                "statFlatMPPoolMod",
                                                "statFlatMagicDamageMod",
                                                "statFlatMovementSpeedMod",
                                                "statFlatPhysicalDamageMod",
                                                "statFlatSpellBlockMod",
                                                "statPercentAttackSpeedMod",
                                                "statPercentLifeStealMod",
                                                "statPercentMovementSpeedMod"])

df_items_spark = df_items_spark.withColumnRenamed("into", "itemsInto")

df_items_spark = df_items_spark.drop("gold", "stats")
df_items_spark = df_items_spark.withColumn("itemsInto", explode("itemsInto"))
df_items_spark = df_items_spark.na.fill(0, subset=["itemsInto"])
df_items_spark = df_items_spark.withColumn("itemsInto", col("itemsInto").cast("long"))

df_items_spark = df_items_spark.withColumn('plaintext', when(col('plaintext') == '', "None").otherwise(col('plaintext')))

df_items_spark = df_items_spark.withColumn("description", regexp_replace("description", "<[^>]+>", ""))
df_items_spark = df_items_spark.withColumn("description", regexp_replace(col("description"), "[\n\t\r]", " "))

df_items_spark = df_items_spark.withColumnRenamed("id", "itemId")

df_items_spark = df_items_spark.withColumn("itemId", col("itemId").cast("long"))


In [14]:
df_items_spark.show(1,truncate=False)

+------+-----+-------------+-----------------------------+---------+--------+---------+--------+----------------+---------------------+-----------------+------------------+-----------------+----------------------+------------------------+-------------------------+---------------------+-------------------------+-----------------------+---------------------------+
|itemId|name |description  |plaintext                    |itemsInto|goldBase|goldTotal|goldSell|statFlatArmorMod|statFlatCritChanceMod|statFlatHPPoolMod|statFlatHPRegenMod|statFlatMPPoolMod|statFlatMagicDamageMod|statFlatMovementSpeedMod|statFlatPhysicalDamageMod|statFlatSpellBlockMod|statPercentAttackSpeedMod|statPercentLifeStealMod|statPercentMovementSpeedMod|
+------+-----+-------------+-----------------------------+---------+--------+---------+--------+----------------+---------------------+-----------------+------------------+-----------------+----------------------+------------------------+-------------------------+------

**LOADING DATA TO DATA WAREHOUSE (AWS REDSHIFT)**

In [ ]:
from configparser import ConfigParser

parser = ConfigParser()
parser.read("cluster.conf")
db_credentials = parser["db_credentials"]
aws_credentials = parser["aws"]


CREATE TABLES

In [40]:
create_dim_summoner = """
    CREATE TABLE IF NOT EXISTS dim_summoner(
        puuid VARCHAR(255) NOT NULL,
        summonerId VARCHAR(255) NOT NULL,
        accountId VARCHAR(255) NOT NULL,
        leagueId VARCHAR(255) NOT NULL,
        gameName VARCHAR(255) NOT NULL,
        tagLine VARCHAR(255) NOT NULL,
        summonerLevel INT NOT NULL,
        queueType VARCHAR(255),
        tier VARCHAR(255),
        rank VARCHAR(255),
        leaguePoints INT,
        wins INT,
        losses INT,
        veteran INT,
        inactive INT,
        freshBlood INT,
        hotStreak INT,
        revisionDate TIMESTAMP,
        PRIMARY KEY (puuid)
    );
    """
    
    
create_dim_champions="""
    CREATE TABLE IF NOT EXISTS dim_champions(
        championId INT NOT NULL,
        name VARCHAR(255) NOT NULL,
        title VARCHAR(255),
        blurb VARCHAR(2048),
        tags VARCHAR(255),
        partype VARCHAR(255),
        attack INT,
        defense INT,
        magic INT,
        difficulty INT,
        hp INT,
        hpPerLevel INT,
        mp INT,
        mpPerLevel FLOAT,
        moveSpeed INT,
        armor INT,
        armorPerLevel FLOAT,
        spellBlock INT,
        spellBlockPerLevel FLOAT,
        attackRange INT,
        hpRegen FLOAT,
        hpRegenPerLevel FLOAT,
        mpRegen FLOAT,
        mpRegenPerLevel FLOAT,
        crit INT,
        critPerLevel INT,
        attackDamage INT,
        attackDamagePerLevel FLOAT,
        attackSpeed FLOAT,
        attackSpeedPerLevel FLOAT,
        PRIMARY KEY (championId)
    );
    """

create_dim_champions_mastery="""
    CREATE TABLE IF NOT EXISTS dim_champions_mastery(
        puuid VARCHAR(255) NOT NULL,
        championId INT NOT NULL,
        championLevel INT,
        championPoints INT,
        lastPlayTime TIMESTAMP,
        FOREIGN KEY(puuid) REFERENCES dim_summoner(puuid),
        FOREIGN KEY(championId) REFERENCES dim_champions(championId)
    );
    """
    
create_dim_matches="""
    CREATE TABLE IF NOT EXISTS dim_matches(
        puuid VARCHAR(255) NOT NULL,
        matchId VARCHAR(255) NOT NULL,
        gameId INT NOT NULL,
        teamId INT,
        lane VARCHAR(255),
        role VARCHAR(255),
        teamPosition VARCHAR(255),
        individualPosition VARCHAR(255),
        endOfGameResult VARCHAR(255),
        gameCreation TIMESTAMP,
        gameStartTimestamp TIMESTAMP,
        gameEndTimestamp TIMESTAMP,
        gameMode VARCHAR(255),
        gameName VARCHAR(255),
        gameType VARCHAR(255),
        mapId INT,
        platformId VARCHAR(255),
        queueId INT,
        gameVersion VARCHAR(255),
        PRIMARY KEY(matchId),
        FOREIGN KEY(puuid) REFERENCES dim_summoner(puuid)
    );
    """
    
create_dim_items="""
    CREATE TABLE IF NOT EXISTS dim_items(
        itemId INT NOT NULL,
        name VARCHAR(255),
        description VARCHAR(2048),
        plaintext VARCHAR(255),
        itemsInto INT,
        goldBase INT,
        goldTotal INT,
        goldSell INT,
        statFlatArmorMod INT,
        statFlatCritChanceMod FLOAT,
        statFlatHPPoolMod INT,
        statFlatHPRegenMod FLOAT,
        statFlatMPPoolMod INT,
        statFlatMagicDamageMod INT,
        statFlatMovementSpeedMod INT,
        statFlatPhysicalDamageMod INT,
        statFlatSpellBlockMod INT,
        statPercentAttackSpeedMod FLOAT,
        statPercentLifeStealMod FLOAT,
        statPercentMovementSpeedMod FLOAT,
        PRIMARY KEY(itemId)
    );"""
    
create_fact_matches="""
    CREATE TABLE IF NOT EXISTS fact_matches(
        puuid VARCHAR(255),
        matchId VARCHAR(255),
        championId INT,
        gameDuration FLOAT,
        win INT,
        banChampionId INT,
        kills INT,
        deaths INT,
        assists INT,
        champLevel INT,
        totalMinionsKilled INT,
        goldEarned INT,
        goldSpent INT,
        visionScore INT,
        dragonKills INT,
        turretKills INT,
        doubleKills INT,
        tripleKills INT,
        quadraKills INT,
        pentaKills INT,
        killingSprees INT,
        inhibitorKills INT,
        nexusKills INT,
        inhibitorTakedowns INT,
        inhibitorsLost INT,
        objectivesStolen INT,
        item0 INT,
        item1 INT,
        item2 INT,
        item3 INT,
        item4 INT,
        item5 INT,
        item6 INT,
        FOREIGN KEY(puuid) REFERENCES dim_summoner(puuid),
        FOREIGN KEY(matchId) REFERENCES dim_matches(matchId),
        FOREIGN KEY(championId) REFERENCES dim_champions(championId),
        FOREIGN KEY(banChampionId) REFERENCES dim_champions(championId),
        FOREIGN KEY(item0) REFERENCES dim_items(itemId),
        FOREIGN KEY(item1) REFERENCES dim_items(itemId),
        FOREIGN KEY(item2) REFERENCES dim_items(itemId),
        FOREIGN KEY(item3) REFERENCES dim_items(itemId),
        FOREIGN KEY(item4) REFERENCES dim_items(itemId),
        FOREIGN KEY(item5) REFERENCES dim_items(itemId),
        FOREIGN KEY(item6) REFERENCES dim_items(itemId)
    );
    """

In [ ]:
import redshift_connector

conn = redshift_connector.connect(
        host=db_credentials["host"],
        database=db_credentials["database"],
        user = db_credentials["user"],
        password=db_credentials["password"],
        port=db_credentials["port"]
    )

try:
    with conn.cursor() as cursor:
        cursor.execute(create_dim_summoner)
        cursor.execute(create_dim_champions)
        cursor.execute(create_dim_champions_mastery)
        cursor.execute(create_dim_matches)
        cursor.execute(create_dim_items)
        cursor.execute(create_fact_matches) 
        
        conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Exception: {e}")
finally:
    conn.close()

LOAD

In [ ]:
try:
    with conn.cursor() as cursor:
        cursor.execute("TRUNCATE TABLE fact_matches;")
        cursor.execute("TRUNCATE TABLE dim_matches;")
        cursor.execute("TRUNCATE TABLE dim_items;")
        cursor.execute("TRUNCATE TABLE dim_champions_mastery;")
        cursor.execute("TRUNCATE TABLE dim_champions;")
        cursor.execute("TRUNCATE TABLE dim_summoner;")
        
    fact_matches.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "fact_matches") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
            
    dim_matches.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "dim_matches") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
                
    df_items_spark.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "dim_items") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
                    
    df_champions_mastery_spark.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "dim_champions_mastery") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
                
    df_champions_spark.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "dim_champions") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
                    
    df_summoner_spark.write \
                .format("jdbc") \
                .option("url", f"jdbc:redshift://{db_credentials['host']}:{db_credentials['port']}/{db_credentials['database']}") \
                .option("dbtable", "dim_summoner") \
                .option("aws_iam_role", aws_credentials["aws_iam_role"]) \
                .option("tempdir", aws_credentials["temp_s3_bucket"]) \
                .option("user", db_credentials['user']) \
                .option("password", db_credentials['password']) \
                .option("driver", "com.amazon.redshift.jdbc42.Driver") \
                .mode("append") \
                .save()
except Exception as e:
        print(f"Exception: {e}")
finally:
    conn.close()